In [5]:
import subprocess
subprocess.run(['pip', 'install', '-q', 'sqlalchemy==1.3.9', 'ipython-sql', 'prettytable', 'pandas'])

import sqlite3
import pandas as pd

# ── Setup: connect to DB and load data ──────────────────────────
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method="multi")

cur.execute("DROP TABLE IF EXISTS SPACEXTABLE;")
cur.execute("CREATE TABLE SPACEXTABLE AS SELECT * FROM SPACEXTBL WHERE Date IS NOT NULL;")
con.commit()

print("Columns:", df.columns.tolist())

def run_query(title, query):
    print(f"\n{'='*60}\n  {title}\n{'='*60}")
    result = pd.read_sql_query(query, con)
    print(result.to_string(index=False))
    return result

# ── Task 1: Unique launch sites ──────────────────────────────────
run_query(
    "TASK 1: Unique Launch Sites",
    "SELECT DISTINCT Launch_Site FROM SPACEXTABLE;"
)

# ── Task 2: 5 records where launch site begins with 'CCA' ────────
run_query(
    "TASK 2: 5 Records with Launch Sites starting with 'CCA'",
    "SELECT * FROM SPACEXTABLE WHERE Launch_Site LIKE 'CCA%' LIMIT 5;"
)

# ── Task 3: Total payload mass for NASA (CRS) ────────────────────
run_query(
    "TASK 3: Total Payload Mass for NASA (CRS)",
    "SELECT SUM(PAYLOAD_MASS__KG_) AS Total_Payload_Mass_kg FROM SPACEXTABLE WHERE Customer LIKE '%NASA (CRS)%';"
)

# ── Task 4: Average payload mass for booster F9 v1.1 ────────────
run_query(
    "TASK 4: Average Payload Mass for Booster F9 v1.1",
    "SELECT AVG(PAYLOAD_MASS__KG_) AS Avg_Payload_Mass_kg FROM SPACEXTABLE WHERE Booster_Version = 'F9 v1.1';"
)

# ── Task 5: First successful ground pad landing date ─────────────
run_query(
    "TASK 5: First Successful Ground Pad Landing Date",
    "SELECT MIN(Date) AS First_Ground_Pad_Success FROM SPACEXTABLE WHERE Landing_Outcome = 'Success (ground pad)';"
)

# ── Task 6: Boosters with drone ship success & payload 4000-6000 ─
run_query(
    "TASK 6: Boosters - Drone Ship Success & Payload 4000-6000 kg",
    """
    SELECT Booster_Version FROM SPACEXTABLE
    WHERE Landing_Outcome = 'Success (drone ship)'
      AND PAYLOAD_MASS__KG_ > 4000
      AND PAYLOAD_MASS__KG_ < 6000;
    """
)

# ── Task 7: Count of successful and failed mission outcomes ───────
run_query(
    "TASK 7: Total Successful and Failed Mission Outcomes",
    """
    SELECT Mission_Outcome, COUNT(*) AS Count
    FROM SPACEXTABLE
    GROUP BY Mission_Outcome
    ORDER BY Count DESC;
    """
)

# ── Task 8: Booster versions with maximum payload mass ───────────
run_query(
    "TASK 8: Booster Versions with Maximum Payload Mass",
    """
    SELECT Booster_Version, PAYLOAD_MASS__KG_
    FROM SPACEXTABLE
    WHERE PAYLOAD_MASS__KG_ = (SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTABLE);
    """
)

# ── Task 9: Failed drone ship landings in 2015 by month ──────────
run_query(
    "TASK 9: Failed Drone Ship Landings in 2015 by Month",
    """
    SELECT
        CASE substr(Date, 6, 2)
            WHEN '01' THEN 'January'   WHEN '02' THEN 'February'
            WHEN '03' THEN 'March'     WHEN '04' THEN 'April'
            WHEN '05' THEN 'May'       WHEN '06' THEN 'June'
            WHEN '07' THEN 'July'      WHEN '08' THEN 'August'
            WHEN '09' THEN 'September' WHEN '10' THEN 'October'
            WHEN '11' THEN 'November'  WHEN '12' THEN 'December'
        END AS Month,
        Landing_Outcome,
        Booster_Version,
        Launch_Site
    FROM SPACEXTABLE
    WHERE Landing_Outcome = 'Failure (drone ship)'
      AND substr(Date, 0, 5) = '2015';
    """
)

# ── Task 10: Ranked landing outcomes 2010-06-04 to 2017-03-20 ────
run_query(
    "TASK 10: Landing Outcome Counts (2010-06-04 to 2017-03-20)",
    """
    SELECT Landing_Outcome, COUNT(*) AS Outcome_Count
    FROM SPACEXTABLE
    WHERE Date BETWEEN '2010-06-04' AND '2017-03-20'
    GROUP BY Landing_Outcome
    ORDER BY Outcome_Count DESC;
    """
)

print("\n" + "="*60)
print("  All 10 Tasks Completed!")
print("="*60)

con.close()

Columns: ['Date', 'Time (UTC)', 'Booster_Version', 'Launch_Site', 'Payload', 'PAYLOAD_MASS__KG_', 'Orbit', 'Customer', 'Mission_Outcome', 'Landing_Outcome']

  TASK 1: Unique Launch Sites
 Launch_Site
 CCAFS LC-40
 VAFB SLC-4E
  KSC LC-39A
CCAFS SLC-40

  TASK 2: 5 Records with Launch Sites starting with 'CCA'
      Date Time (UTC) Booster_Version Launch_Site                                                       Payload  PAYLOAD_MASS__KG_     Orbit        Customer Mission_Outcome     Landing_Outcome
2010-06-04   18:45:00  F9 v1.0  B0003 CCAFS LC-40                          Dragon Spacecraft Qualification Unit                  0       LEO          SpaceX         Success Failure (parachute)
2010-12-08   15:43:00  F9 v1.0  B0004 CCAFS LC-40 Dragon demo flight C1, two CubeSats, barrel of Brouere cheese                  0 LEO (ISS) NASA (COTS) NRO         Success Failure (parachute)
2012-05-22    7:44:00  F9 v1.0  B0005 CCAFS LC-40                                         Dragon demo flight 